In [65]:
import numpy as np
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium

In [41]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 

In [42]:
df = pd.read_html(str(table))[0]

In [43]:
df = df.loc[df['Borough'] != 'Not assigned',:]

In [45]:
df.reset_index(drop=True,inplace=True)

In [46]:
df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [47]:
df.shape

(103, 3)

## Get Coordinates 

In [51]:
coordinates = pd.read_csv('./Geospatial_Coordinates.csv')

In [56]:
df = df.join(coordinates.set_index('Postal Code'),on='Postal Code',how='inner')

In [58]:
df.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [66]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Create a map and markers


In [68]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### Exploring neighborhoods in Toronto


In [69]:
borough_names = list(df.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [72]:
# create a new DataFrame with only boroughs that contain the word Toronto
df = df[df['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(df.shape)
df.head()

(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [73]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [74]:
CLIENT_ID = 'N5WIP1LRUNONK13GA2J003N1D1LLE2HWHGUDPLOPCPXNG1I3' # your Foursquare ID
CLIENT_SECRET = 'BI0T433A5KCX0LCMRNPT1HSJRSTXFCJC4VSQSOAYCBHC1MFM'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: N5WIP1LRUNONK13GA2J003N1D1LLE2HWHGUDPLOPCPXNG1I3
CLIENT_SECRET:BI0T433A5KCX0LCMRNPT1HSJRSTXFCJC4VSQSOAYCBHC1MFM


In [76]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Postal Code'], df['Borough'], 
                                                  df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=ZYLC4Q3I000O4R32DVJWJJTOTHCGC4O02TXYEPLDAS211SPQ&client_secret=OPKHF1MTRWKRHVR2DAV0IT1IK2H2XZDXJYTCNHVY5L44T55H&v=20180605 \
     &ll=43.653963,-79.387207&radius=500&limit=100".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

### Convert venues list to new dataframe

In [77]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1716, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Textile Museum of Canada,43.654396,-79.386500,Art Museum
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cafe Plenty,43.654571,-79.389450,Café
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Japango,43.655268,-79.385165,Sushi Restaurant


### Check how many venues are returned

In [78]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,44,44,44,44,44,44
M4K,East Toronto,"The Danforth West, Riverdale",44,44,44,44,44,44
M4L,East Toronto,"India Bazaar, The Beaches West",44,44,44,44,44,44
M4M,East Toronto,Studio District,44,44,44,44,44,44
M4N,Central Toronto,Lawrence Park,44,44,44,44,44,44
M4P,Central Toronto,Davisville North,44,44,44,44,44,44
M4R,Central Toronto,North Toronto West,44,44,44,44,44,44
M4S,Central Toronto,Davisville,44,44,44,44,44,44
M4T,Central Toronto,"Moore Park, Summerhill East",44,44,44,44,44,44


### Analysing each area

In [79]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1716, 37)


,PostalCode,Borough,Neighborhoods,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,Café,...,Poke Place,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Smoke Shop,Sushi Restaurant,University,Vegetarian / Vegan Restaurant
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [80]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped


(39, 37)


,PostalCode,Borough,Neighborhoods,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,Café,...,Poke Place,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Smoke Shop,Sushi Restaurant,University,Vegetarian / Vegan Restaurant
0,M4E,East Toronto,The Beaches,0.068182,0.022727,0.022727,0.022727,0.022727,0.022727,0.068182,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727
1,M4K,East Toronto,"The Danforth West, Riverdale",0.068182,0.022727,0.022727,0.022727,0.022727,0.022727,0.068182,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.068182,0.022727,0.022727,0.022727,0.022727,0.022727,0.068182,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727
3,M4M,East Toronto,Studio District,0.068182,0.022727,0.022727,0.022727,0.022727,0.022727,0.068182,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727
4,M4N,Central Toronto,Lawrence Park,0.068182,0.022727,0.022727,0.022727,0.022727,0.022727,0.068182,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727
5,M4P,Central Toronto,Davisville North,0.068182,0.022727,0.022727,0.022727,0.022727,0.022727,0.068182,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727
6,M4R,Central Toronto,North Toronto West,0.068182,0.022727,0.022727,0.022727,0.022727,0.022727,0.068182,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727
7,M4S,Central Toronto,Davisville,0.068182,0.022727,0.022727,0.022727,0.022727,0.022727,0.068182,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.068182,0.022727,0.022727,0.022727,0.022727,0.022727,0.068182,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.068182,0.022727,0.022727,0.022727,0.022727,0.022727,0.068182,...,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.022727,0.045455,0.022727,0.022727


### New dataframe and display the top 10 venues for each PostalCode

In [81]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted


(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
3,M4M,East Toronto,Studio District,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
4,M4N,Central Toronto,Lawrence Park,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
5,M4P,Central Toronto,Davisville North,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
6,M4R,Central Toronto,North Toronto West,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
7,M4S,Central Toronto,Davisville,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
8,M4T,Central Toronto,"Moore Park, Summerhill East",Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop


### CLUSTERING

In [82]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/Users/dsuarezsouto/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:972: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [88]:
#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = df.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="Postal Code")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!


(39, 16)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop


In [89]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
21,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
22,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
23,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
25,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
26,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
27,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
20,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
28,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop


### Visualizing the clusters

In [91]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + \
                                                                                 list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
21,Central Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
22,West Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
23,Central Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
24,Central Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
25,West Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
26,Central Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
27,Downtown Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
20,Central Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
28,West Toronto,0,Coffee Shop,Art Gallery,Café,Japanese Restaurant,Sushi Restaurant,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop
